# Preprocessing

In [3]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion fundamental frequency

In [1]:
import numpy as np
import librosa

def calculate_f0(audio_signal, sample_rate=16000, frame_length=2048, hop_length=512, fmin=75, fmax=300):
    """
    Berechnet die Fundamental Frequency (F0) eines Audiosignals mit der pYIN-Methode.

    Quelle:
        Mauch, M., & Dixon, S. (2014). pYIN: A fundamental frequency estimator using probabilistic threshold distributions.
        IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP).

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).
        frame_length (int): Länge eines Frames in Samples (Standard: 2048).
        hop_length (int): Schrittweite zwischen Frames in Samples (Standard: 512).
        fmin (int): Minimale erwartete Frequenz in Hz (Standard: 75 Hz für Sprache).
        fmax (int): Maximale erwartete Frequenz in Hz (Standard: 300 Hz für Sprache).

    Returns:
        float: Durchschnittliche Fundamental Frequency (F0) in Hz.
    """
    # Berechnung der Grundfrequenz mit pYIN (eine verbesserte Version von YIN)
    f0_values, voiced_flag, _ = librosa.pyin(
        audio_signal, fmin=fmin, fmax=fmax, sr=sample_rate, frame_length=frame_length, hop_length=hop_length
    )

    # Nur die Werte berücksichtigen, die als "voiced" erkannt wurden (keine NaN-Werte)
    f0_values = f0_values[~np.isnan(f0_values)]

    # Durchschnittliche Grundfrequenz berechnen
    average_f0 = np.mean(f0_values) if len(f0_values) > 0 else 0.0
    return average_f0

# Test

In [4]:
import os

# F0 für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)
        
        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)
        
        # F0 berechnen
        f0_value = calculate_f0(audio_signal)
        
        print(f"Fundamental Frequency (F0) ({file_name}): {f0_value:.2f} Hz")

Fundamental Frequency (F0) (_noise_pink.wav): 0.00 Hz
Fundamental Frequency (F0) (_noise_white.wav): 0.00 Hz
Fundamental Frequency (F0) (_signal_constant.wav): 115.48 Hz
Fundamental Frequency (F0) (_signal_silence.wav): 0.00 Hz
Fundamental Frequency (F0) (_signal_sine.wav): 219.61 Hz
Fundamental Frequency (F0) (_snr_03.wav): 145.36 Hz
Fundamental Frequency (F0) (_snr_10.wav): 147.72 Hz
Fundamental Frequency (F0) (_snr_20.wav): 149.41 Hz
Fundamental Frequency (F0) (example1.wav): 112.49 Hz
Fundamental Frequency (F0) (example2.wav): 108.56 Hz
Fundamental Frequency (F0) (example3.wav): 114.34 Hz
